### Install Packages

In [1]:
!pip install "datasets>=2.14.0" "torch>=2.0.0" --quiet
!pip install bitsandbytes evaluate huggingface_hub transformers --quiet
!pip install -U peft --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 11.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is

### Imports

In [2]:
from kagglehub import model_download, KaggleDatasetAdapter, load_dataset
from kaggle_secrets import UserSecretsClient
import copy
import evaluate
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, set_seed, TrainingArguments

from datasets import Dataset

2025-11-10 04:13:42.257515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762748022.436645      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762748022.488805      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Set Seed

In [3]:
SEED = 42

### Choose Device

In [4]:
if torch.cuda.is_available():
    device = "cuda"
    print(f"Using CUDA GPU: {torch.cuda.get_device_name()}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")

else:
    device = "cpu"
    print("Using CPU - you will need to use a GPU to train models")

Using CUDA GPU: Tesla P100-PCIE-16GB
GPU memory: 17.1GB


### Load Dataset

In [5]:
TEST_SET_PATH = "bingxuanchia/dsa4213-medquad-processed-dataset"

# Load and verify dataset
medquad_test = load_dataset(
    KaggleDatasetAdapter.HUGGING_FACE,
    TEST_SET_PATH,
    "test.csv",
)

/tmp/ipykernel_19/3778376697.py:4: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  medquad_test = load_dataset(


In [6]:
# Convert to pandas for grouping
medquad_test_pd = medquad_test.to_pandas()

# Group by question and aggregate answers into lists
medquad_test_pd_grouped = medquad_test_pd.groupby("question")["answer"].apply(list).reset_index()

# Convert back to Hugging Face dataset
medquad_test = Dataset.from_pandas(medquad_test_pd_grouped)

### Configurations

In [7]:
hf_token = UserSecretsClient().get_secret("HF_TOKEN")
os.environ["HF_TOKEN"] = hf_token

### Helpers

In [8]:
def get_response_to_query(generator, question):
    set_seed(SEED)

    messages = [{"role": "user", "content": question}]
    output = generator(messages, max_new_tokens=256, return_full_text=False)[0]

    return output["generated_text"]

In [9]:
def generate_responses(batch, generator, max_new_tokens=256):
    # set seed before generation
    set_seed(42)
    # Format each question into a chat message list
    formatted_inputs = [[{"role": "user", "content": q}] for q in batch["question"]]
    
    # Generate batched outputs
    outputs = generator(
        formatted_inputs,
        max_new_tokens=max_new_tokens,
        return_full_text=False,
        batch_size=len(formatted_inputs)
    )
    
    # Extract the generated text from each result
    batch["responses"] = [out[0]["generated_text"] for out in outputs]
    return batch


def get_testset_with_responses(testset, model_names):
    results_dataset = copy.deepcopy(testset)

    for model_name in model_names:
        print(f"Generating responses for {model_name}...")
        # Quantization configurations
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16
        )
        
        if model_name == "HuggingFaceTB/SmolLM2-1.7B-Instruct-Quantized":  
            quantized_model = AutoModelForCausalLM.from_pretrained(
                "HuggingFaceTB/SmolLM2-1.7B-Instruct",
                quantization_config=quantization_config,
            )

            tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-1.7B-Instruct")
            
            generator = pipeline(
                "text-generation",
                model=quantized_model,
                tokenizer=tokenizer,
            )
        else:
            quantized_model = AutoModelForCausalLM.from_pretrained(
                model_name,
                quantization_config=quantization_config,
            )
            
            generator = pipeline(
                "text-generation",
                model=model_name,
                device='cuda'
            )

        # Use dataset.map for batched parallel processing
        test_with_responses = results_dataset.map(
            lambda batch: generate_responses(batch, generator),
            batched=True,
            batch_size=32,   # adjust for GPU memory
        )

        model_label = model_name.split("/")[-1]
        model_col_name = f"{model_label}_responses"
        results_dataset = test_with_responses.rename_column("responses", model_col_name)

    return results_dataset

### Response Generation

In [10]:
# Set model names
final_model_names = [
    "HuggingFaceTB/SmolLM2-1.7B-Instruct-Quantized",
    "Jiahao123/SmolLM2-1.7B-Instruct-MediLite-QA-Rank8-Quantized-HighLR", # Best finetuned model
    "Jiahao123/medilite-grpo-v1" # RL model
]

In [ ]:
final_results_dataset = get_testset_with_responses(
    testset=medquad_test,
    model_names=final_model_names
)

# NOTE: uncomment to push when running
HF_REPO_ID = "Cowboygarage/MediLite-QA-Response-Evaluation"
final_results_dataset.push_to_hub(HF_REPO_ID)

Generating responses for HuggingFaceTB/SmolLM2-1.7B-Instruct-Quantized...


config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

model.safetensors:   0%|          | 0.00/3.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

...

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md:   0%|          | 0.00/532 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/Cowboygarage/MediLite-QA-Response-Evaluation/commit/fa6be8b96e3abe41ce17285531e00080097c1b6d', commit_message='Upload dataset', commit_description='', oid='fa6be8b96e3abe41ce17285531e00080097c1b6d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Cowboygarage/MediLite-QA-Response-Evaluation', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Cowboygarage/MediLite-QA-Response-Evaluation'), pr_revision=None, pr_num=None)